In [1]:
####input your credentials here
consumer_key = '*******************************************'
consumer_secret = '*******************************************'
access_token = '*******************************************'
access_token_secret = '*******************************************'

In [2]:
import tweepy

In [3]:
# Preparamos la autenticación
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)#creamos un autentificador que contiene nuestras claves

# Preparamos el módulo api de Tweepy (que es el que nos va a ayudar a hacer las "preguntas" a la API)
api = tweepy.API(auth)

In [4]:
import datetime
startDate= datetime.datetime(2020,11,21,0,0,0)
endDate = datetime.datetime(2020,11,23,0,0,0)

In [16]:
# Importamos PyMongo
from pymongo import MongoClient

# Creamos una conexión con la BBDD
client = MongoClient()
           
# Usamos una base de datos llamada test
db = client.test

In [ ]:
for tweet in tweepy.Cursor(api.search,q=("#AMAs2020 OR #AMAs"),count=100,
                   tweet_mode= 'extended',start = startDate, until = endDate).items(40000):
    db.tweets.insert_one(tweet._json)

In [40]:
import networkx as nx

G = nx.DiGraph()


for result in db.tweets.find():

            uid = result['user']['screen_name']
            G.add_node(uid) # si ya existe lo omite solo
            
            # Hay más casuísticas, pero por ejemplo, con el retweeted_status (es decir, un user ha RT a otro)
            if 'retweeted_status' in result:
                if G.has_edge(uid, result['retweeted_status']['user']['screen_name']):
                    G[uid][result['retweeted_status']['user']['screen_name']]['weight'] += 1.0
                else:
                    G.add_edge(uid, result['retweeted_status']['user']['screen_name'], weight = 1.0) 
            
for result in db.tweets.find():
        #vemos si hay personas mencionadas en el tuit
        if 'entities' in result:
            try: #si no hay nadie mencionado simplemente lo omite
                for i in range(len(result['entities']['user_mentions'][1]['screen_name'])):
                    G.add_node(result['entities']['user_mentions'][0]['screen_name'])
                    G.add_node(result['entities']['user_mentions'][i]['screen_name'])
                    if G.has_edge(result['entities']['user_mentions'][0]['screen_name'], result['entities']['user_mentions'][1]['screen_name'][i]):
                        G[result['entities']['user_mentions'][0]['screen_name']][result['entities']['user_mentions'][1]['screen_name'][i]]['weight'] += 1.0
                    else:
                        G.add_edge(result['entities']['user_mentions'][0]['screen_name'], result['entities']['user_mentions'][1]['screen_name'][i], weight = 1.0) 
            except: 
                continue
                
for result in db.tweets.find():
    uid = result['user']['screen_name']
    #si es una respuesta a un twit
    if result['in_reply_to_screen_name'] != 'null':
        G.add_node(result['in_reply_to_screen_name'])
        if G.has_edge(uid, result['in_reply_to_screen_name']):
            G[uid][result['in_reply_to_screen_name']]['weight'] += 1.0
        else:
            G.add_edge(uid, result['in_reply_to_screen_name'], weight = 1.0)



In [42]:
nx.write_graphml(G, "AMAs.graphml")


In [ ]:
len(G)

In [31]:
G.clear()